In [1]:
import requests
import re
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import sys
import matplotlib.pyplot as plt
from collections import Counter
import xml.etree.ElementTree as ET
from typing import List, Dict, Any, Optional
import duckdb
import unicodedata
import importlib
import numpy as np
import csv

from pathlib import Path
import re
import pandas as pd

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
import grela_helpers

In [3]:
conn = grela_helpers.make_connection()

In [4]:
#grela_extraction_helpers.conn.close()
# importlib.reload(grela_extraction_helpers)

In [5]:
# test with one specific multiword phrase
target_canonical     =  "auaritia" # "lapis philosophorum"
target_relemmatized  =  "auaritia" # "lapis philosophus"
# 1) phase 1: spans only (whole GreLa, but capped)
grela_helpers.create_target_spans_table(
    conn,
    table_name="temp_target_spans_table",
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    emlap_only=False,
    max_hits=1_000_000,   # safeguard
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()

(17248,)

In [7]:
# 2) phase 2: build detailed KWIC for first N hits
df = grela_helpers.kwic_from_spans(
    conn,
    spans_table="temp_target_spans_table",
    window=10,
    include_tokens=True,
    max_hits=100_000,     # or None, if you know it's safe
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
df.sample(10)

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
8378,[avaritiae],canonical,lemma,[auaritia],[173680087],[10],[7],cc_8921,cc_8921_1890,cc_8921_1889,cc_8921_1890,terram dicit Dominus A minore quippe usque ad ...,"[{'token_id': 173680077, 'token_text': 'terram...",A minore quippe usque ad majorem omnes avariti...,"[{'token_id': 173680080, 'token_text': 'A', 'l...",Rabanus Maurus,Commentaria in Jeremiam,780,856
10280,[avaritiam],canonical,lemma,[auaritia],[225627159],[10],[19],cc_12033,cc_12033_272318,cc_12033_272318,cc_12033_272318,repromissam benedictionem hanc paratam esse si...,"[{'token_id': 225627149, 'token_text': 'reprom...",Et sic accipitur avaritia II ad Cor IX praepar...,"[{'token_id': 225627140, 'token_text': 'Et', '...",Thomas Aquinas,Summa Theologiae,1225,1274
14196,[avaritiam],canonical,lemma,[auaritia],[225628040],[10],[38],cc_12033,cc_12033_272396,cc_12033_272396,cc_12033_272397,exteriores quae sunt ad usum hominis deputatae...,"[{'token_id': 225628030, 'token_text': 'exteri...",Et secundum hoc peccatum quod est contra Deum ...,"[{'token_id': 225628002, 'token_text': 'Et', '...",Thomas Aquinas,Summa Theologiae,1225,1274
3947,[auaritiam],canonical,lemma,[auaritia],[136371808],[10],[9],cc_12136,cc_12136_7499,cc_12136_7498,cc_12136_7499,". Et quando homo , fide habituatus , peccat pe...","[{'token_id': 136371798, 'token_text': '.', 'l...","Et quando homo , fide habituatus , peccat per ...","[{'token_id': 136371799, 'token_text': 'Et', '...",Raimundus Lullus,Ars generalis ultima,1232,1315
6416,[auaritia],canonical,lemma,[auaritia],[48782366],[10],[1],cc_12021,cc_12021_10665,cc_12021_10664,cc_12021_10666,"insanabilis , ut philosophus dicit in Iu Ethic...","[{'token_id': 48782356, 'token_text': 'insanab...",Ergo auaritia est peccatum mortale et grauissi...,"[{'token_id': 48782365, 'token_text': 'Ergo', ...",Thomas Aquinas,Quaestiones disputatae de malo,1225,1274
5866,[avaritiam],canonical,lemma,[auaritia],[232403037],[10],[3],cc_7978,cc_7978_124,cc_7978_123,cc_7978_125,tuas amici magis sentiant beneficiis quam inju...,"[{'token_id': 232403027, 'token_text': 'tuas',...",Pecunia non satiat avaritiam sed irritat,"[{'token_id': 232403034, 'token_text': 'Pecuni...",Martinus Braccarensis,De moribus,520,580
7072,[avaritiam],canonical,lemma,[auaritia],[201793373],[10],[40],cc_9428,cc_9428_13037,cc_9428_13037,cc_9428_13038,utilitate subditorum suscipi creditur quando i...,"[{'token_id': 201793363, 'token_text': 'utilit...",Sed subjecti quoque pro eo quod in maximis gra...,"[{'token_id': 201793333, 'token_text': 'Sed', ...",Odo Cluniacensis,Moralia in Job,878,942
1300,[avaritiam],canonical,lemma,[auaritia],[115974908],[10],[13],cc_13080,cc_13080_16112,cc_13080_16112,cc_13080_16112,una cum abscondita conclusione facile audienti...,"[{'token_id': 115974898, 'token_text': 'una', ...",Que quidem verba una cum abscondita conclusion...,"[{'token_id': 115974895, 'token_text': 'Que', ...",Giovanni Boccaccio,Genealogie deorum gentilium libri,1313,1375
3783,[avaritiam],canonical,lemma,[auaritia],[221593063],[10],[6],cc_6951,cc_6951_1729,cc_6951_1728,cc_6951_1729,et qui ferus tristis Et de vobis qui pecuniam ...,"[{'token_id': 221593053, 'token_text': 'et', '...",Et de vobis qui pecuniam et avaritiam devorati...,"[{'token_id': 221593057, 'token_text': 'Et', '...",Ambrosius Mediolanensis,De Tobia,340,397
15051,[avaritia],canonical,lemma,[auaritia],[57443556],[10],[16],cc_12733,cc_12733_45217,cc_12733_45217,cc_12733_45218,falso — quid inquam magno opere potuit Cleomen...,"[{'token_id': 57443546, 'token_text': 'falso',...",— non enim possum quemquam insimulare falso — ...,"[{'token_id': 57443540, 'token_text': '—', 'le...",Cicero,Orationes,-106,-43


In [10]:
df.to_parquet("../data/auaritia.parquet")